In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import argparse
import lfigw.waveform_generator as wfg

In [3]:
from dataclasses import dataclass

@dataclass
class Args:
    event: str
    outdir: str
    generate_reduced_basis: int
    generate_dataset: int
    generate_noisy_test_data: int

In [4]:
args = Args(
    event='GW150914',
    outdir='waveforms',
    generate_reduced_basis=50000,
    generate_dataset=1000000,
    generate_noisy_test_data=5000
)

In [6]:
# check valid gravitational wave event input
gw_event = args.event
valid_gw_events = ['GW150914', 'GW170814']
assert gw_event in valid_gw_events, (
    f"event {gw_event} must be one of {valid_gw_events}"
)

# check save dir
assert os.path.exists(args.outdir)
save_path = os.path.join(args.outdir, gw_event)

wfd = wfg.WaveformDataset(
    spins_aligned=False,
    domain='RB',
    extrinsic_at_train=True
)

wfd.Nrb = 600
wfd.approximant = 'IMRPhenomPv2'

wfd.load_event(f'data/events/{gw_event}/')

wfd.importance_sampling = 'uniform_distance'

# edit priors
wfd.prior['distance'] = [100.0, 1000.0]
wfd.prior['a_1'][1] = 0.88
wfd.prior['a_2'][1] = 0.88

print('Dataset properties')
print('Event', wfd.event)
print(wfd.prior)
print('f_min', wfd.f_min)
print('f_min_psd', wfd.f_min_psd)
print('f_max', wfd.f_max)
print('T', wfd.time_duration)
print('reference time', wfd.ref_time)

# long runtime
wfd.generate_reduced_basis(args.generate_reduced_basis)  # args can specify samples used to generate svd basis
wfd.generate_dataset(args.generate_dataset)  # if basis=None, we generate basis with default number of samples
# wfd.generate_noisy_test_data(args.generate_noisy_test_data)

# save generated waveform data
# wfd.save(save_path)
# wfd.save_train(save_path)
# wfd.save_noisy_test_data(save_path)

print('Program complete.')


  0%|          | 0/50000 [00:00<?, ?it/s]

Dataset properties
Event GW150914
{'mass_1': [10.0, 80.0], 'mass_2': [10.0, 80.0], 'phase': [0.0, 6.283185307179586], 'time': [-0.1, 0.1], 'distance': [100.0, 1000.0], 'chi_1': [-1.0, 1.0], 'chi_2': [-1.0, 1.0], 'a_1': [0.0, 0.88], 'a_2': [0.0, 0.88], 'tilt_1': [0.0, 3.141592653589793], 'tilt_2': [0.0, 3.141592653589793], 'phi_12': [0.0, 6.283185307179586], 'phi_jl': [0.0, 6.283185307179586], 'theta_jn': [0.0, 3.141592653589793], 'psi': [0.0, 3.141592653589793], 'ra': [0.0, 6.283185307179586], 'dec': [-1.5707963267948966, 1.5707963267948966]}
f_min 20.0
f_min_psd 20.0
f_max 1024.0
T 8.0
reference time 1126259462.391
Generating 50000 detector FD waveforms for training reduced basis.
PyCBC PSD from_txt
PyCBC PSD from_txt


100%|██████████| 50000/50000 [02:15<00:00, 367.73it/s]


Generating reduced basis for training detector waveforms
training_data: [[0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j ... 3.4608533e-05+3.641247e-05j
  3.7329941e-05+4.027698e-05j 0.0000000e+00+0.000000e+00j]
 [0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j ... 0.0000000e+00-0.000000e+00j
  0.0000000e+00-0.000000e+00j 0.0000000e+00-0.000000e+00j]
 [0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j ... 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j]
 ...
 [0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j ... 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j]
 [0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j ... 0.0000000e+00+0.000000e+00j
  0.0000000e+00+0.000000e+00j 0.0000000e+00+0.000000e+00j]
 [0.0000000e+00+0.000000e+00

  0%|          | 25/100000 [00:00<06:42, 248.45it/s]

...

PyCBC PSD from_txt
Generating + and x waveforms.


  0%|          | 0/1001 [00:00<?, ?it/s]

Performing additional dataset initialization.
Building time translation matrices.


100%|██████████| 1001/1001 [03:05<00:00,  5.40it/s]


Program complete.
